In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

/home/cmantas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
fname = 'data/news/uci-news-aggregator.csv'
n_most_common_words = 8000
max_len = 130

In [ ]:
data = pd.read_csv(fname, usecols=['TITLE', 'CATEGORY'])
data.CATEGORY.value_counts()

In [5]:
# removed balancing of categories

In [6]:
#One-hot encode the lab
for l, category in enumerate(('e', 'b', 't', 'm')):
    data.loc[data['CATEGORY'] == category, 'LABEL'] = l
print(data['LABEL'][:10])


labels = to_categorical(data['LABEL'], num_classes=4)
print(labels[:10])
if 'CATEGORY' in data.keys():
    data.drop(['CATEGORY'], axis=1)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: LABEL, dtype: float64
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [7]:
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['TITLE'].values)
sequences = tokenizer.texts_to_sequences(data['TITLE'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 75583 unique tokens.


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=42)

In [9]:
epochs = 10
emb_dim = 128
batch_size = 256

In [10]:
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))


((316814, 130), (316814, 4), (105605, 130), (105605, 4))


In [11]:
model = Sequential([
    Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]),
    SpatialDropout1D(0.7),
    LSTM(64, dropout=0.7, recurrent_dropout=0.7),
    Dense(4, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 128)          1024000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 130, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 1,073,668
Trainable params: 1,073,668
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

Train on 253451 samples, validate on 63363 samples
Epoch 1/10
253451/253451 [==============================] - 758s 3ms/step - loss: 0.5678 - acc: 0.7794 - val_loss: 0.2472 - val_acc: 0.9164
Epoch 2/10
253451/253451 [==============================] - 767s 3ms/step - loss: 0.2853 - acc: 0.9031 - val_loss: 0.2210 - val_acc: 0.9247
Epoch 3/10
253451/253451 [==============================] - 815s 3ms/step - loss: 0.2494 - acc: 0.9157 - val_loss: 0.2126 - val_acc: 0.9272
Epoch 4/10
237824/253451 [===========================>..] - ETA: 49s - loss: 0.2332 - acc: 0.9210

KeyboardInterrupt: 